## Compare Convex Solver vs. the GD Solver for the Two-layer ReLU Network

The goal of this notebook is to compare two training approaches for two-layer ReLU neural networks using the entire training dataset:

1. Its exact convex formulation, solved using CVXPY.
2. Traditional backpropagation (BP), using full-batch gradient descent (GD).

We evaluate both methods on two types of synthetic tasks:

- A classification task using the spiral dataset.
- A regression task based on learning the quadratic function $y = x^2$.

In [ ]:
import sys
sys.path.append('..')
from src.baselines.skactive_baseline import *
from src.cpal.cpal import *
from src.cpal.synthetic_data import *
from src.cpal.solve_cvxnn import *

### 1. Classification on Synthetic Spiral

In [ ]:
# 1. Generate synthetic spiral data
X_all, y_all, X, y, Xacc, yacc = generate_spiral_data()

In [ ]:
# 2. Generate hyperplane arrangement
dmat= generate_hyperplane_arrangement(X,P=1000)
_, num_neurons = dmat.shape

In [ ]:
# 3. Convex solve exact 2-layer neural network
Uopt1, Uopt2, obj_val, status = solve_two_layer_cvx(X, y, dmat, beta=1e-5, task_type='c')

In [ ]:
# Plot the corresponding CVX decision boundary
plot_decision_boundary(
    X=X,
    y=y,
    X_test=Xacc,
    y_test=yacc,
    selected_indices=None,
    name='Decision Boundary (Convex)',
    Uopt1v=Uopt1,
    Uopt2v=Uopt2,
    plot_type='cvx'
)

In [ ]:
# 4. Compare with GD solver via BP
m = num_neurons
U, w, obj_bp = train_bp_two_layer_relu(X, y, m, cvx_opt=obj_val)

In [ ]:
# Plot the corresponding BP decision boundary
plot_decision_boundary(
    X=X,
    y=y,
    X_test=Xacc,
    y_test=yacc,
    name='Decision Boundary (BP)',
    U = U,
    w = w,
    plot_type='bp'
)


In [ ]:
evaluate_model_performance(task = 'c', X_test = Xacc, y_test = yacc, Uopt1 = Uopt1, Uopt2 = Uopt2, U = U, w = w)

### 2. Regression

In [ ]:
# 1. Generate synthetic quadratic regression data
X_all, y_all, X, y, Xacc, yacc = generate_quadratic_regression()

In [ ]:
# 2. Generate hyperplane arrangement
dmat= generate_hyperplane_arrangement(X,P=2000)
_, num_neurons = dmat.shape

In [ ]:
# 3. convex solve exact 2-layer neural network
Uopt1, Uopt2, obj_val, status = solve_two_layer_cvx(X, y, dmat)

In [ ]:
# 4. bp comparison
m = num_neurons
U_bp, w_bp, obj_bp = train_bp_two_layer_relu(X, y, m, cvx_opt=obj_val)

In [ ]:
# 5. Compare performance
y_true = X_all[:,:-1] ** 2 # for quadratic regression
evaluate_model_performance(task = 'r', X_all = X_all, y_true = y_true, X = X, y = y, X_test = Xacc, y_test = yacc, Uopt1 = Uopt1, Uopt2 = Uopt2, U = U_bp, w = w_bp)